# Converting Data for Visualization

Although we've managed to extract a few examples of both dabs and tposes, it's now time to figure out what our data looks like. 

The easiest way to manipulate and visualize data in Python is via tools like Pandas and Seaborn. 

But first, we'll need to convert our numpy raw arrays into something that's a bit more readable. So let's do that by converting them into labeled CSV files.

In [1]:
import numpy as np
np.random.seed(1337)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dabDataset = np.load('data/dabs.npy')
tposeDataset = np.load('data/tposes.npy')
otherDataset = np.load('data/other.npy')

In [3]:
dabDataset[0]

array([[5.8832416e+02, 2.9433704e+02, 7.2265184e-01],
       [5.8239331e+02, 3.5126093e+02, 8.0205584e-01],
       [5.0984329e+02, 3.4919385e+02, 7.5316119e-01],
       [4.1784265e+02, 3.1985785e+02, 8.1164622e-01],
       [3.6101605e+02, 2.9243521e+02, 8.0296052e-01],
       [6.5091376e+02, 3.6097537e+02, 6.4161348e-01],
       [6.3724268e+02, 2.7274924e+02, 7.8188539e-01],
       [4.9614203e+02, 2.4154723e+02, 8.3243752e-01],
       [5.4315808e+02, 6.4114813e+02, 4.4807938e-01],
       [4.8636816e+02, 6.2938318e+02, 3.6906898e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [6.0191382e+02, 6.4702966e+02, 3.8946095e-01],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00],
       [5.7648334e+02, 2.7475522e+02, 6.1822432e-01],
       [6.0389270e+02, 2.8454663e+02, 4.1854110e-01],
       [5.5686536e+02, 2.6891223e+02, 2.7014270e-01],
       [6.1959991e+02, 2.924

In [4]:
dabDataset[0].shape

(25, 3)

# Adding our Labels

Our labels come from the [BODY_25 Pose Output format](https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/output.md#pose-output-format-body_25) available at the repo. 

We can tell because when we looked at each of our poses, we saw a `dataset[0].shape` of 25. This matches the number of labels below.

In [5]:
labels = ["Nose", "Neck", "RShoulder", "RElbow", "RWrist", "LShoulder", "LElbow",
 "LWrist", "MidHip", "RHip", "RKnee", "RAnkle", "LHip", "LKnee", "LAnkle",
 "REye", "LEye", "REar", "LEar", "LBigToe", "LSmallToe", "LHeel", "RBigToe",
 "RSmallToe", "RHeel", "Background"]

Each of our labels comes as an `X`, `Y`, and `Confidence`. Let's add those labels and flatten this array for our CSV file:

In [6]:
properLabels = []
for label in labels:
    properLabels.append(label + 'X')
    properLabels.append(label + 'Y')
    properLabels.append(label + 'Confidence')

In [7]:
import csv

with open('data/dabs.csv', 'w+') as dabcsv:
    dabwriter = csv.writer(dabcsv, delimiter=',')
    dabwriter.writerow(properLabels)
    for cell in dabDataset:
        dabwriter.writerow(cell.flatten())
        
with open('data/tposes.csv', 'w+') as tposecsv:
    tposewriter = csv.writer(tposecsv, delimiter=',')
    tposewriter.writerow(properLabels)
    for cell in tposeDataset:
        tposewriter.writerow(cell.flatten())
        
with open('data/other.csv', 'w+') as othercsv:
    otherwriter = csv.writer(othercsv, delimiter=',')
    otherwriter.writerow(properLabels)
    for cell in otherDataset:
        otherwriter.writerow(cell.flatten())

## Sanity Checking our Data

We can now open up our CSV files and see what they look like. How many samples did we collect? Is it enough? 

Once we check, we can hop on to the next step, bringing all the data into a single format and file for training.

## Creating a Labeled Dataset for Training and Testing

Now that we've got our data (mostly) sorted out, we need to convert it into a set. 

We'll use `0` for `other` poses, `1` for `dabs`, and `2` for `tposes`.



In [8]:
labels = np.zeros(len(otherDataset))
labels = np.append(labels, np.full((len(dabDataset)), 1))
labels = np.append(labels, np.full((len(tposeDataset)), 2))
print(labels)
print("%i total examples for training." % len(labels))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2.]
56 total examples for training.


In [9]:
dataset = np.append(otherDataset, dabDataset, axis=0)
dataset = np.append(dataset, tposeDataset, axis=0)
print(dataset)

[[[488.3213     147.51425      0.83340967]
  [494.22372    284.5734       0.8012297 ]
  [386.4863     270.83716      0.66853976]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[515.7737     112.18818      0.83487195]
  [478.48004    274.7029       0.8005627 ]
  [368.77948    257.2105       0.6782713 ]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 [[547.1316     112.15151      0.79948723]
  [464.79065    268.88403      0.73338044]
  [360.98135    243.43745      0.62600124]
  ...
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]
  [  0.           0.           0.        ]]

 ...

 [[509.97504    257.06958      0.892523  ]
  [460.9663     351.17117      0.7867987 ]
  [372.75305    333.54434      0.6111988 ]
  ...
  [  0.           0.           0.        ]
  [  0.           

In [10]:
dataset.shape

(56, 25, 3)

In [11]:
dataset[:,:,1] / 1280

array([[0.11524551, 0.22232297, 0.21159153, ..., 0.        , 0.        ,
        0.        ],
       [0.08764701, 0.21461165, 0.2009457 , ..., 0.        , 0.        ,
        0.        ],
       [0.08761837, 0.21006565, 0.19018552, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.20083562, 0.2743525 , 0.26058152, ..., 0.        , 0.        ,
        0.        ],
       [0.2238348 , 0.29272196, 0.27588812, ..., 0.        , 0.        ,
        0.        ],
       [0.2100658 , 0.28972745, 0.2758514 , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [12]:
# now, let's shuffle labels and the array, the same way
from sklearn.utils import shuffle
X, y = shuffle(dataset, labels)
# now let's label them for 'one hot'
from keras.utils.np_utils import to_categorical
y = to_categorical(y, 3)
print(y.shape[1])

Using TensorFlow backend.


3


In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import SGD

In [14]:
X = X.reshape(len(X), 75)
dataset[0]

array([[4.88321289e+02, 1.47514252e+02, 8.33409667e-01],
       [4.94223724e+02, 2.84573395e+02, 8.01229715e-01],
       [3.86486298e+02, 2.70837158e+02, 6.68539762e-01],
       [3.37498718e+02, 4.31440033e+02, 8.06459844e-01],
       [2.76727325e+02, 5.92155334e+02, 6.95721209e-01],
       [6.01926575e+02, 2.96297577e+02, 6.77372575e-01],
       [6.17621460e+02, 4.47154175e+02, 8.15527081e-01],
       [6.33207092e+02, 6.13721497e+02, 7.50288665e-01],
       [4.49166534e+02, 6.23515259e+02, 3.33123893e-01],
       [3.68768433e+02, 6.15664124e+02, 2.96909660e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.29464417e+02, 6.35266663e+02, 3.00662249e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.72626495e+02, 1.25848732e+02, 7.93599010e-01],
       [5.09897217e+02, 1.25915306e+02, 8.75047982e-01],
       [4.47158661e+02, 1.31820

In [15]:
opt = SGD(lr=0.005)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(75,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer=opt, #'Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=200,batch_size=50)

Epoch 1/200
56/56 [==============================] - 0s 2ms/step - loss: 11.8007 - acc: 0.2679
Epoch 2/200
56/56 [==============================] - 0s 49us/step - loss: 11.7810 - acc: 0.2679
Epoch 3/200
56/56 [==============================] - 0s 49us/step - loss: 8.9225 - acc: 0.4464
Epoch 4/200
56/56 [==============================] - 0s 44us/step - loss: 8.3469 - acc: 0.4821
Epoch 5/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 6/200
56/56 [==============================] - 0s 48us/step - loss: 8.3469 - acc: 0.4821
Epoch 7/200
56/56 [==============================] - 0s 41us/step - loss: 8.6347 - acc: 0.4643
Epoch 8/200
56/56 [==============================] - 0s 57us/step - loss: 8.6347 - acc: 0.4643
Epoch 9/200
56/56 [==============================] - 0s 67us/step - loss: 8.6347 - acc: 0.4643
Epoch 10/200
56/56 [==============================] - 0s 54us/step - loss: 8.3469 - acc: 0.4821
Epoch 11/200
56/56 [============================

56/56 [==============================] - 0s 49us/step - loss: 8.6347 - acc: 0.4643
Epoch 87/200
56/56 [==============================] - 0s 46us/step - loss: 8.6347 - acc: 0.4643
Epoch 88/200
56/56 [==============================] - 0s 51us/step - loss: 8.6347 - acc: 0.4643
Epoch 89/200
56/56 [==============================] - 0s 45us/step - loss: 8.6347 - acc: 0.4643
Epoch 90/200
56/56 [==============================] - 0s 46us/step - loss: 8.6347 - acc: 0.4643
Epoch 91/200
56/56 [==============================] - 0s 42us/step - loss: 8.6347 - acc: 0.4643
Epoch 92/200
56/56 [==============================] - 0s 48us/step - loss: 8.6347 - acc: 0.4643
Epoch 93/200
56/56 [==============================] - 0s 39us/step - loss: 8.3469 - acc: 0.4821
Epoch 94/200
56/56 [==============================] - 0s 50us/step - loss: 8.6347 - acc: 0.4643
Epoch 95/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 96/200
56/56 [==============================] -

56/56 [==============================] - 0s 47us/step - loss: 8.6347 - acc: 0.4643
Epoch 171/200
56/56 [==============================] - 0s 45us/step - loss: 8.6347 - acc: 0.4643
Epoch 172/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 173/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 174/200
56/56 [==============================] - 0s 45us/step - loss: 8.6347 - acc: 0.4643
Epoch 175/200
56/56 [==============================] - 0s 46us/step - loss: 8.6347 - acc: 0.4643
Epoch 176/200
56/56 [==============================] - 0s 50us/step - loss: 8.6347 - acc: 0.4643
Epoch 177/200
56/56 [==============================] - 0s 47us/step - loss: 8.6347 - acc: 0.4643
Epoch 178/200
56/56 [==============================] - 0s 43us/step - loss: 8.6347 - acc: 0.4643
Epoch 179/200
56/56 [==============================] - 0s 59us/step - loss: 8.6347 - acc: 0.4643
Epoch 180/200
56/56 [=======================

# Cleaning up data further

Looking at our accuracy, it looks like we need to better prepare and nomalize our data. 

Or maybe we need to try a different optimizer. Here, I've tried both `SGD` and `Adam`, and saw no improvement.

In [16]:
# let's fit x and y to 0 - 1, get rid of confidence and try again
X, y = shuffle(dataset, labels)
y = to_categorical(y, 3)
print(X.shape)
X[:,:,0] = X[:,:,0] / 720 # I think the dimensions are 1280 x 720 ?
X[:,:,1] = X[:,:,1] / 1280  # let's see?
X = X[:,:,1:]
print(X.shape)
X = X.reshape(56, 50)      # we got rid of confidence percentage
print(X.shape)

(56, 25, 3)
(56, 25, 2)
(56, 50)


In [17]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=200,batch_size=25)

Epoch 1/200
56/56 [==============================] - 0s 3ms/step - loss: 1.1181 - acc: 0.3214
Epoch 2/200
56/56 [==============================] - 0s 85us/step - loss: 1.1080 - acc: 0.4107
Epoch 3/200
56/56 [==============================] - 0s 62us/step - loss: 1.0874 - acc: 0.3571
Epoch 4/200
56/56 [==============================] - 0s 73us/step - loss: 1.0482 - acc: 0.4286
Epoch 5/200
56/56 [==============================] - 0s 89us/step - loss: 1.0640 - acc: 0.4107
Epoch 6/200
56/56 [==============================] - 0s 64us/step - loss: 1.0018 - acc: 0.4107
Epoch 7/200
56/56 [==============================] - 0s 59us/step - loss: 1.0232 - acc: 0.5536
Epoch 8/200
56/56 [==============================] - 0s 83us/step - loss: 1.0465 - acc: 0.4286
Epoch 9/200
56/56 [==============================] - 0s 104us/step - loss: 1.0455 - acc: 0.4821
Epoch 10/200
56/56 [==============================] - 0s 90us/step - loss: 1.0816 - acc: 0.4464
Epoch 11/200
56/56 [=============================

56/56 [==============================] - 0s 73us/step - loss: 0.4853 - acc: 0.7500
Epoch 87/200
56/56 [==============================] - 0s 71us/step - loss: 0.4310 - acc: 0.8571
Epoch 88/200
56/56 [==============================] - 0s 73us/step - loss: 0.4120 - acc: 0.8393
Epoch 89/200
56/56 [==============================] - 0s 83us/step - loss: 0.4127 - acc: 0.8571
Epoch 90/200
56/56 [==============================] - 0s 81us/step - loss: 0.4381 - acc: 0.8393
Epoch 91/200
56/56 [==============================] - 0s 81us/step - loss: 0.4103 - acc: 0.8571
Epoch 92/200
56/56 [==============================] - 0s 74us/step - loss: 0.4100 - acc: 0.8393
Epoch 93/200
56/56 [==============================] - 0s 73us/step - loss: 0.3548 - acc: 0.8750
Epoch 94/200
56/56 [==============================] - 0s 74us/step - loss: 0.4244 - acc: 0.8750
Epoch 95/200
56/56 [==============================] - 0s 63us/step - loss: 0.4012 - acc: 0.8393
Epoch 96/200
56/56 [==============================] -

56/56 [==============================] - 0s 70us/step - loss: 0.2678 - acc: 0.9107
Epoch 171/200
56/56 [==============================] - 0s 62us/step - loss: 0.2800 - acc: 0.8929
Epoch 172/200
56/56 [==============================] - 0s 61us/step - loss: 0.3129 - acc: 0.7857
Epoch 173/200
56/56 [==============================] - 0s 76us/step - loss: 0.2324 - acc: 0.9107
Epoch 174/200
56/56 [==============================] - 0s 66us/step - loss: 0.2621 - acc: 0.8750
Epoch 175/200
56/56 [==============================] - 0s 70us/step - loss: 0.2920 - acc: 0.8750
Epoch 176/200
56/56 [==============================] - 0s 64us/step - loss: 0.3364 - acc: 0.8214
Epoch 177/200
56/56 [==============================] - 0s 84us/step - loss: 0.2525 - acc: 0.9107
Epoch 178/200
56/56 [==============================] - 0s 60us/step - loss: 0.2810 - acc: 0.8571
Epoch 179/200
56/56 [==============================] - 0s 70us/step - loss: 0.2379 - acc: 0.9107
Epoch 180/200
56/56 [=======================

In [18]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=500,batch_size=25)

Epoch 1/500
56/56 [==============================] - 0s 4ms/step - loss: 1.1755 - acc: 0.2857
Epoch 2/500
56/56 [==============================] - 0s 83us/step - loss: 1.1476 - acc: 0.3036
Epoch 3/500
56/56 [==============================] - 0s 63us/step - loss: 1.1500 - acc: 0.2500
Epoch 4/500
56/56 [==============================] - 0s 60us/step - loss: 1.1591 - acc: 0.3214
Epoch 5/500
56/56 [==============================] - 0s 82us/step - loss: 1.1428 - acc: 0.3750
Epoch 6/500
56/56 [==============================] - 0s 77us/step - loss: 1.1098 - acc: 0.3393
Epoch 7/500
56/56 [==============================] - 0s 61us/step - loss: 1.1165 - acc: 0.2857
Epoch 8/500
56/56 [==============================] - 0s 57us/step - loss: 1.1031 - acc: 0.3929
Epoch 9/500
56/56 [==============================] - 0s 67us/step - loss: 1.0903 - acc: 0.3750
Epoch 10/500
56/56 [==============================] - 0s 86us/step - loss: 1.0967 - acc: 0.3393
Epoch 11/500
56/56 [==============================

Epoch 86/500
56/56 [==============================] - 0s 65us/step - loss: 0.6595 - acc: 0.6429
Epoch 87/500
56/56 [==============================] - 0s 72us/step - loss: 0.6975 - acc: 0.5893
Epoch 88/500
56/56 [==============================] - 0s 71us/step - loss: 0.6693 - acc: 0.6607
Epoch 89/500
56/56 [==============================] - 0s 71us/step - loss: 0.6520 - acc: 0.7143
Epoch 90/500
56/56 [==============================] - 0s 69us/step - loss: 0.7347 - acc: 0.6250
Epoch 91/500
56/56 [==============================] - 0s 60us/step - loss: 0.7615 - acc: 0.6429
Epoch 92/500
56/56 [==============================] - ETA: 0s - loss: 0.5417 - acc: 0.800 - 0s 82us/step - loss: 0.6381 - acc: 0.7321
Epoch 93/500
56/56 [==============================] - 0s 78us/step - loss: 0.6086 - acc: 0.7321
Epoch 94/500
56/56 [==============================] - 0s 73us/step - loss: 0.6893 - acc: 0.6429
Epoch 95/500
56/56 [==============================] - 0s 71us/step - loss: 0.6588 - acc: 0.6607
Ep

56/56 [==============================] - 0s 71us/step - loss: 0.4366 - acc: 0.7321
Epoch 171/500
56/56 [==============================] - 0s 75us/step - loss: 0.3971 - acc: 0.7857
Epoch 172/500
56/56 [==============================] - 0s 75us/step - loss: 0.3896 - acc: 0.8214
Epoch 173/500
56/56 [==============================] - 0s 66us/step - loss: 0.4260 - acc: 0.6607
Epoch 174/500
56/56 [==============================] - 0s 63us/step - loss: 0.3747 - acc: 0.7857
Epoch 175/500
56/56 [==============================] - 0s 80us/step - loss: 0.4116 - acc: 0.7679
Epoch 176/500
56/56 [==============================] - 0s 73us/step - loss: 0.3785 - acc: 0.8214
Epoch 177/500
56/56 [==============================] - 0s 68us/step - loss: 0.3909 - acc: 0.8214
Epoch 178/500
56/56 [==============================] - 0s 66us/step - loss: 0.4703 - acc: 0.7500
Epoch 179/500
56/56 [==============================] - 0s 78us/step - loss: 0.4011 - acc: 0.8393
Epoch 180/500
56/56 [=======================

56/56 [==============================] - 0s 77us/step - loss: 0.3264 - acc: 0.8036
Epoch 255/500
56/56 [==============================] - 0s 69us/step - loss: 0.2954 - acc: 0.8571
Epoch 256/500
56/56 [==============================] - 0s 112us/step - loss: 0.3095 - acc: 0.8214
Epoch 257/500
56/56 [==============================] - 0s 79us/step - loss: 0.3491 - acc: 0.7500
Epoch 258/500
56/56 [==============================] - 0s 65us/step - loss: 0.3050 - acc: 0.8036
Epoch 259/500
56/56 [==============================] - 0s 86us/step - loss: 0.3773 - acc: 0.7679
Epoch 260/500
56/56 [==============================] - 0s 72us/step - loss: 0.3345 - acc: 0.7857
Epoch 261/500
56/56 [==============================] - 0s 62us/step - loss: 0.3123 - acc: 0.8036
Epoch 262/500
56/56 [==============================] - 0s 63us/step - loss: 0.3070 - acc: 0.8036
Epoch 263/500
56/56 [==============================] - 0s 77us/step - loss: 0.3079 - acc: 0.8393
Epoch 264/500
56/56 [======================

56/56 [==============================] - 0s 73us/step - loss: 0.2902 - acc: 0.8393
Epoch 339/500
56/56 [==============================] - 0s 98us/step - loss: 0.3000 - acc: 0.8393
Epoch 340/500
56/56 [==============================] - 0s 73us/step - loss: 0.3063 - acc: 0.8571
Epoch 341/500
56/56 [==============================] - 0s 76us/step - loss: 0.2626 - acc: 0.8750
Epoch 342/500
56/56 [==============================] - 0s 82us/step - loss: 0.2693 - acc: 0.8214
Epoch 343/500
56/56 [==============================] - 0s 72us/step - loss: 0.2581 - acc: 0.8571
Epoch 344/500
56/56 [==============================] - 0s 70us/step - loss: 0.2955 - acc: 0.8393
Epoch 345/500
56/56 [==============================] - 0s 82us/step - loss: 0.2246 - acc: 0.9107
Epoch 346/500
56/56 [==============================] - 0s 64us/step - loss: 0.2856 - acc: 0.8393
Epoch 347/500
56/56 [==============================] - 0s 72us/step - loss: 0.2447 - acc: 0.8929
Epoch 348/500
56/56 [=======================

56/56 [==============================] - 0s 99us/step - loss: 0.2788 - acc: 0.8393
Epoch 423/500
56/56 [==============================] - 0s 82us/step - loss: 0.2705 - acc: 0.8750
Epoch 424/500
56/56 [==============================] - 0s 78us/step - loss: 0.2407 - acc: 0.8571
Epoch 425/500
56/56 [==============================] - 0s 91us/step - loss: 0.2025 - acc: 0.9107
Epoch 426/500
56/56 [==============================] - 0s 78us/step - loss: 0.2208 - acc: 0.9286
Epoch 427/500
56/56 [==============================] - 0s 78us/step - loss: 0.2895 - acc: 0.8393
Epoch 428/500
56/56 [==============================] - 0s 84us/step - loss: 0.2260 - acc: 0.8750
Epoch 429/500
56/56 [==============================] - 0s 81us/step - loss: 0.2327 - acc: 0.8929
Epoch 430/500
56/56 [==============================] - 0s 83us/step - loss: 0.2579 - acc: 0.8750
Epoch 431/500
56/56 [==============================] - 0s 104us/step - loss: 0.1970 - acc: 0.9643
Epoch 432/500
56/56 [======================

# Adding More Data and Beginning Data Augmentation

We've started to improve upon our inital approach. Our accuracy is now approaching 0.76. But now, let's add more examples, and switch between a `train` and `test` dataset.

I've run the data collection program again, and this time doubled the examples we have to train on. Now that we better understand our data, we can speed up it's import and cleaning.

In [19]:
dabDataset = np.load('data/more-dabs.npy')
tposeDataset = np.load('data/more-tposes.npy')
otherDataset = np.load('data/more-other.npy')
labels1 = np.zeros(len(otherDataset))
labels1 = np.append(labels1, np.full((len(dabDataset)), 1))
labels1 = np.append(labels1, np.full((len(tposeDataset)), 2))
print(labels1)
print("%i total new samples" % len(labels1))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.
 2. 2. 2.]
171 total new samples


In [20]:
dataset1 = np.append(otherDataset, dabDataset, axis=0)
dataset1 = np.append(dataset1, tposeDataset, axis=0)
X1, y1 = shuffle(dataset1, labels1)
y1 = to_categorical(y1, 3)
print(X1.shape)
X1[:,:,0] = X1[:,:,0] / 720 # I think the dimensions are 1280 x 720 ?
X1[:,:,1] = X1[:,:,1] / 1280  # let's see?
X1 = X1[:,:,1:]
print(X1.shape)
X1 = X1.reshape(len(X1), 50)      # we got rid of confidence percentage
print(X1.shape)

(171, 25, 3)
(171, 25, 2)
(171, 50)


In [21]:
opt = SGD(lr=0.01)
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(50,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X1, y1, epochs=2000,batch_size=25)

Epoch 1/2000
171/171 [==============================] - 0s 1ms/step - loss: 1.1371 - acc: 0.4094
Epoch 2/2000
171/171 [==============================] - 0s 49us/step - loss: 1.0732 - acc: 0.4795
Epoch 3/2000
171/171 [==============================] - 0s 52us/step - loss: 1.1044 - acc: 0.3626
Epoch 4/2000
171/171 [==============================] - 0s 53us/step - loss: 1.0715 - acc: 0.3860
Epoch 5/2000
171/171 [==============================] - 0s 48us/step - loss: 0.9873 - acc: 0.5263
Epoch 6/2000
171/171 [==============================] - 0s 52us/step - loss: 0.9624 - acc: 0.5322
Epoch 7/2000
171/171 [==============================] - 0s 47us/step - loss: 0.9664 - acc: 0.5029
Epoch 8/2000
171/171 [==============================] - 0s 51us/step - loss: 0.9284 - acc: 0.5848
Epoch 9/2000
171/171 [==============================] - 0s 38us/step - loss: 0.8881 - acc: 0.5731
Epoch 10/2000
171/171 [==============================] - 0s 44us/step - loss: 0.8896 - acc: 0.5731
Epoch 11/2000
171/17

171/171 [==============================] - 0s 45us/step - loss: 0.1441 - acc: 0.9591
Epoch 84/2000
171/171 [==============================] - 0s 43us/step - loss: 0.1269 - acc: 0.9591
Epoch 85/2000
171/171 [==============================] - 0s 51us/step - loss: 0.1232 - acc: 0.9766
Epoch 86/2000
171/171 [==============================] - 0s 53us/step - loss: 0.1393 - acc: 0.9649
Epoch 87/2000
171/171 [==============================] - 0s 40us/step - loss: 0.1428 - acc: 0.9415
Epoch 88/2000
171/171 [==============================] - 0s 60us/step - loss: 0.1361 - acc: 0.9591
Epoch 89/2000
171/171 [==============================] - 0s 47us/step - loss: 0.1016 - acc: 0.9766
Epoch 90/2000
171/171 [==============================] - 0s 58us/step - loss: 0.1265 - acc: 0.9591
Epoch 91/2000
171/171 [==============================] - 0s 40us/step - loss: 0.1230 - acc: 0.9474
Epoch 92/2000
171/171 [==============================] - 0s 49us/step - loss: 0.1240 - acc: 0.9532
Epoch 93/2000
171/171 [=

171/171 [==============================] - 0s 54us/step - loss: 0.0365 - acc: 0.9883
Epoch 165/2000
171/171 [==============================] - 0s 51us/step - loss: 0.0265 - acc: 1.0000
Epoch 166/2000
171/171 [==============================] - 0s 53us/step - loss: 0.0430 - acc: 0.9883
Epoch 167/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0535 - acc: 0.9766
Epoch 168/2000
171/171 [==============================] - 0s 50us/step - loss: 0.0433 - acc: 0.9883
Epoch 169/2000
171/171 [==============================] - 0s 48us/step - loss: 0.0281 - acc: 1.0000
Epoch 170/2000
171/171 [==============================] - 0s 55us/step - loss: 0.0451 - acc: 0.9942
Epoch 171/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0600 - acc: 0.9883
Epoch 172/2000
171/171 [==============================] - 0s 46us/step - loss: 0.0322 - acc: 0.9942
Epoch 173/2000
171/171 [==============================] - 0s 48us/step - loss: 0.0446 - acc: 0.9883
Epoch 174/2000


171/171 [==============================] - 0s 47us/step - loss: 0.0234 - acc: 0.9942
Epoch 247/2000
171/171 [==============================] - 0s 45us/step - loss: 0.0293 - acc: 0.9942
Epoch 248/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0153 - acc: 0.9942
Epoch 249/2000
171/171 [==============================] - 0s 39us/step - loss: 0.0572 - acc: 0.9883
Epoch 250/2000
171/171 [==============================] - 0s 46us/step - loss: 0.0248 - acc: 0.9883
Epoch 251/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0170 - acc: 0.9942
Epoch 252/2000
171/171 [==============================] - 0s 56us/step - loss: 0.0146 - acc: 1.0000
Epoch 253/2000
171/171 [==============================] - 0s 52us/step - loss: 0.0169 - acc: 1.0000
Epoch 254/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0203 - acc: 0.9942
Epoch 255/2000
171/171 [==============================] - 0s 50us/step - loss: 0.0109 - acc: 1.0000
Epoch 256/2000


171/171 [==============================] - 0s 48us/step - loss: 0.0083 - acc: 1.0000
Epoch 328/2000
171/171 [==============================] - 0s 51us/step - loss: 0.0240 - acc: 0.9883
Epoch 329/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0268 - acc: 0.9942
Epoch 330/2000
171/171 [==============================] - 0s 45us/step - loss: 0.0094 - acc: 0.9942
Epoch 331/2000
171/171 [==============================] - 0s 46us/step - loss: 0.0476 - acc: 0.9825
Epoch 332/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0149 - acc: 0.9942
Epoch 333/2000
171/171 [==============================] - 0s 51us/step - loss: 0.0099 - acc: 1.0000
Epoch 334/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0210 - acc: 0.9883
Epoch 335/2000
171/171 [==============================] - 0s 64us/step - loss: 0.0084 - acc: 1.0000
Epoch 336/2000
171/171 [==============================] - 0s 49us/step - loss: 0.0198 - acc: 0.9883
Epoch 337/2000


...

171/171 [==============================] - 0s 54us/step - loss: 0.0174 - acc: 0.9942
Epoch 1622/2000
171/171 [==============================] - 0s 53us/step - loss: 0.0162 - acc: 0.9942
Epoch 1623/2000
171/171 [==============================] - 0s 42us/step - loss: 0.0111 - acc: 0.9942
Epoch 1624/2000
171/171 [==============================] - 0s 45us/step - loss: 0.0026 - acc: 1.0000
Epoch 1625/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0053 - acc: 1.0000
Epoch 1626/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0017 - acc: 1.0000
Epoch 1627/2000
171/171 [==============================] - 0s 43us/step - loss: 0.0014 - acc: 1.0000
Epoch 1628/2000
171/171 [==============================] - 0s 43us/step - loss: 0.0027 - acc: 1.0000
Epoch 1629/2000
171/171 [==============================] - 0s 49us/step - loss: 5.6839e-04 - acc: 1.0000
Epoch 1630/2000
171/171 [==============================] - 0s 50us/step - loss: 0.0101 - acc: 0.9942
Ep

171/171 [==============================] - 0s 50us/step - loss: 2.9063e-04 - acc: 1.0000
Epoch 1702/2000
171/171 [==============================] - 0s 42us/step - loss: 8.8858e-04 - acc: 1.0000
Epoch 1703/2000
171/171 [==============================] - 0s 44us/step - loss: 0.0021 - acc: 1.0000
Epoch 1704/2000
171/171 [==============================] - 0s 41us/step - loss: 0.0173 - acc: 0.9883
Epoch 1705/2000
171/171 [==============================] - 0s 52us/step - loss: 2.4480e-04 - acc: 1.0000
Epoch 1706/2000
171/171 [==============================] - 0s 43us/step - loss: 0.0045 - acc: 1.0000
Epoch 1707/2000
171/171 [==============================] - 0s 58us/step - loss: 0.0044 - acc: 1.0000
Epoch 1708/2000
171/171 [==============================] - 0s 59us/step - loss: 0.0162 - acc: 0.9942
Epoch 1709/2000
171/171 [==============================] - 0s 40us/step - loss: 0.0012 - acc: 1.0000
Epoch 1710/2000
171/171 [==============================] - 0s 41us/step - loss: 0.0055 - acc: 1

171/171 [==============================] - 0s 41us/step - loss: 3.9914e-04 - acc: 1.0000
Epoch 1781/2000
171/171 [==============================] - 0s 49us/step - loss: 4.0888e-04 - acc: 1.0000
Epoch 1782/2000
171/171 [==============================] - 0s 47us/step - loss: 2.6013e-04 - acc: 1.0000
Epoch 1783/2000
171/171 [==============================] - 0s 56us/step - loss: 0.0185 - acc: 0.9942
Epoch 1784/2000
171/171 [==============================] - 0s 42us/step - loss: 3.2764e-04 - acc: 1.0000
Epoch 1785/2000
171/171 [==============================] - 0s 58us/step - loss: 3.7510e-04 - acc: 1.0000
Epoch 1786/2000
171/171 [==============================] - 0s 53us/step - loss: 8.1681e-04 - acc: 1.0000
Epoch 1787/2000
171/171 [==============================] - 0s 43us/step - loss: 0.0031 - acc: 1.0000
Epoch 1788/2000
171/171 [==============================] - 0s 50us/step - loss: 3.2651e-04 - acc: 1.0000
Epoch 1789/2000
171/171 [==============================] - 0s 47us/step - loss:

171/171 [==============================] - 0s 39us/step - loss: 1.6221e-04 - acc: 1.0000
Epoch 1860/2000
171/171 [==============================] - 0s 46us/step - loss: 1.9505e-04 - acc: 1.0000
Epoch 1861/2000
171/171 [==============================] - 0s 42us/step - loss: 5.5714e-04 - acc: 1.0000
Epoch 1862/2000
171/171 [==============================] - 0s 52us/step - loss: 6.6762e-04 - acc: 1.0000
Epoch 1863/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0014 - acc: 1.0000
Epoch 1864/2000
171/171 [==============================] - 0s 51us/step - loss: 6.2752e-04 - acc: 1.0000
Epoch 1865/2000
171/171 [==============================] - 0s 56us/step - loss: 1.0035e-04 - acc: 1.0000
Epoch 1866/2000
171/171 [==============================] - 0s 46us/step - loss: 0.0019 - acc: 1.0000
Epoch 1867/2000
171/171 [==============================] - 0s 40us/step - loss: 1.9108e-04 - acc: 1.0000
Epoch 1868/2000
171/171 [==============================] - 0s 46us/step - loss:

171/171 [==============================] - 0s 49us/step - loss: 0.0078 - acc: 0.9942
Epoch 1940/2000
171/171 [==============================] - 0s 59us/step - loss: 0.0069 - acc: 1.0000
Epoch 1941/2000
171/171 [==============================] - 0s 48us/step - loss: 0.0016 - acc: 1.0000
Epoch 1942/2000
171/171 [==============================] - 0s 49us/step - loss: 0.0129 - acc: 0.9942
Epoch 1943/2000
171/171 [==============================] - 0s 45us/step - loss: 0.0028 - acc: 1.0000
Epoch 1944/2000
171/171 [==============================] - 0s 41us/step - loss: 0.0294 - acc: 0.9883
Epoch 1945/2000
171/171 [==============================] - 0s 48us/step - loss: 0.0282 - acc: 0.9883
Epoch 1946/2000
171/171 [==============================] - 0s 50us/step - loss: 0.0026 - acc: 1.0000
Epoch 1947/2000
171/171 [==============================] - 0s 55us/step - loss: 0.0268 - acc: 0.9942
Epoch 1948/2000
171/171 [==============================] - 0s 47us/step - loss: 0.0540 - acc: 0.9825
Epoch 

In [22]:
model.predict_classes(X)

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [23]:
y

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0

# It Works! Let's try it out and use our original Data as Test

Finally, after giving it a once over, it looks like the data mostly matches. Let's ensure it's a perfect fit by using keras' `evaluate` to run a test.

In [ ]:
model.test_on_batch(X, y)

# Hooray! Let's save it now!

We use keras' great built in function, `model.save` to save it to disk. 

In our program for inference, we'll use `keras.models.load_model(filepath)` to load it for deteting dab, tpose, or other.

In [24]:
model.save('data/dab-tpose-other.h5')

In [25]:
import keras
modello = keras.models.load_model('data/dab-tpose-other.h5')

In [26]:
dabDataset = np.load('data/test-dabs.npy')
dabDataset[:,:,0] = dabDataset[:,:,0] / 720 # I think the dimensions are 1280 x 720 ?
dabDataset[:,:,1] = dabDataset[:,:,1] / 1280  # let's see?
dabDataset = dabDataset[:,:,1:]
dabDataset = dabDataset.reshape(len(dabDataset), 50)
modello.predict_classes(dabDataset)

array([1, 1, 1, 1, 1, 1])